<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importar-librerias" data-toc-modified-id="Importar-librerias-1">Importar librerias</a></span></li><li><span><a href="#Obtener-datos-de-autenticación" data-toc-modified-id="Obtener-datos-de-autenticación-2">Obtener datos de autenticación</a></span></li><li><span><a href="#Llamada-a-la-API-con-credenciales-y-generación-de-df" data-toc-modified-id="Llamada-a-la-API-con-credenciales-y-generación-de-df-3">Llamada a la API con credenciales y generación de df</a></span></li><li><span><a href="#Guardar-df-como-archivo-csv" data-toc-modified-id="Guardar-df-como-archivo-csv-4">Guardar df como archivo csv</a></span></li></ul></div>

**<center><font size = "6"> Scraping AlphaVantage Data <center>**
***
Documentación: https://www.alphavantage.co/documentation/

### Importar librerias

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import datetime
import time

### Obtener datos de autenticación

In [2]:
load_dotenv()

api_key = os.getenv('AV_APIKEY')

### Llamada a la API con credenciales y generación de df

In [3]:
# Parámetros de la API
params = {
    'function': 'NEWS_SENTIMENT',
    'tickers': 'TSLA',  # Símbolo de Tesla
    'sort': 'RELEVANCE',  # 'EARLIEST' o 'RELEVANCE'
    'limit': '1000',
    'apikey': api_key,
}

url = 'https://www.alphavantage.co/query'

# Fecha de inicio y final en formato YYYYMMDDTHHMM
start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Dividir el rango de fechas en períodos de 3 meses
date_ranges = [start_date + datetime.timedelta(days=x)
               for x in range(0, (end_date-start_date).days, 90)]

# Crear listas vacías para almacenar los datos que queremos extraer
titles = []
urls = []
time_published = []
authors = []
summaries = []
sources = []
category_within_sources = []
source_domains = []
overall_sentiment_scores = []
overall_sentiment_labels = []
ticker_relevance_scores = []
ticker_sentiment_scores = []
ticker_sentiment_labels = []

In [4]:
# Iterar a través de los rangos de fechas y hacer una solicitud para cada uno
for i in range(len(date_ranges)-1):
    params['time_from'] = date_ranges[i].strftime('%Y%m%dT%H%M')
    params['time_to'] = date_ranges[i+1].strftime('%Y%m%dT%H%M')

    # Realiza la solicitud GET a la API
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Verificar si data está vacío
        if not data or "Information" in data:
            print(
                f'No se encontraron artículos en el rango de fechas desde {params["time_from"]} hasta {params["time_to"]}.')
            continue
    else:
        print('Error en la solicitud a la API de Alpha Vantage.')
        continue

    # Iterar a través de los elementos de noticias
    for item in data["feed"]:
        titles.append(item["title"])
        urls.append(item["url"])
        time_published.append(item["time_published"])
        authors.append(", ".join(item["authors"]))
        summaries.append(item["summary"])
        sources.append(item["source"])
        category_within_sources.append(item["category_within_source"])
        source_domains.append(item["source_domain"])
        overall_sentiment_scores.append(item["overall_sentiment_score"])
        overall_sentiment_labels.append(item["overall_sentiment_label"])

        # Verificar si el campo "ticker_sentiment" existe y si el ticker es "TSLA"
        ticker_sentiments = item.get("ticker_sentiment", [])
        tsla_sentiment = next(
            (sentiment for sentiment in ticker_sentiments if sentiment.get("ticker") == "TSLA"), None)
        if tsla_sentiment:
            ticker_relevance_scores.append(
                tsla_sentiment.get("relevance_score", None))
            ticker_sentiment_scores.append(
                tsla_sentiment.get("ticker_sentiment_score", None))
            ticker_sentiment_labels.append(
                tsla_sentiment.get("ticker_sentiment_label", None))
        else:
            # Si no es "TSLA", añadir valores nulos
            ticker_relevance_scores.append(None)
            ticker_sentiment_scores.append(None)
            ticker_sentiment_labels.append(None)

    # Esperar antes de hacer la siguiente solicitud para evitar ser bloqueado por la API
    time.sleep(60)

# Crear DataFrame con los datos extraídos
df = pd.DataFrame({
    "Title": titles,
    "URL": urls,
    "Time Published": time_published,
    "Authors": authors,
    "Summary": summaries,
    "Source": sources,
    "Category Within Source": category_within_sources,
    "Source Domain": source_domains,
    "Overall Sentiment Score": overall_sentiment_scores,
    "Overall Sentiment Label": overall_sentiment_labels,
    "Ticker Relevance Score": ticker_relevance_scores,
    "Ticker Sentiment Score": ticker_sentiment_scores,
    "Ticker Sentiment Label": ticker_sentiment_labels,
})

No se encontraron artículos en el rango de fechas desde 20210101T0000 hasta 20210401T0000.
No se encontraron artículos en el rango de fechas desde 20210401T0000 hasta 20210630T0000.
No se encontraron artículos en el rango de fechas desde 20210630T0000 hasta 20210928T0000.
No se encontraron artículos en el rango de fechas desde 20210928T0000 hasta 20211227T0000.


In [5]:
df

,Title,URL,Time Published,Authors,Summary,Source,Category Within Source,Source Domain,Overall Sentiment Score,Overall Sentiment Label,Ticker Relevance Score,Ticker Sentiment Score,Ticker Sentiment Label
0,Covid Stops Work at Tesla Plant. But the Stock...,https://www.barrons.com/articles/tesla-tsla-st...,20220316T113506,Al Root,Tesla Plant Halted by Covid. The Stock Rises B...,Barrons,n/a,www.barrons.com,-0.525969,Bearish,0.998682,-0.525969,Bearish
1,Tesla Pauses Bond Sale. Stock Investors Don't ...,https://www.barrons.com/articles/tesla-stock-b...,20220317T142700,Al Root,Tesla Pauses Bond Sale. What It Means for the ...,Barrons,n/a,www.barrons.com,-0.251192,Somewhat-Bearish,0.993371,-0.251192,Somewhat-Bearish
2,Tesla Pauses Bond Sale. Stock Investors Don't ...,https://www.barrons.com/articles/tesla-pauses-...,20220317T115555,Al Root,Tesla has paused a $1 billion asset backed sec...,Barrons,n/a,www.barrons.com,-0.237554,Somewhat-Bearish,0.993371,-0.237554,Somewhat-Bearish
3,Tesla Stock Has Been on a Tear. The Reasons Wh...,https://www.barrons.com/articles/tesla-stock-t...,20220326T005544,Al Root,A Tesla Model 3 Plaid can go from 0 to 60 in j...,Barrons,n/a,www.barrons.com,0.197687,Somewhat-Bullish,0.989159,0.197687,Somewhat-Bullish
4,Tesla Stock Has Been on a Tear. Deliveries Can...,https://www.barrons.com/articles/tesla-stock-t...,20220326T005432,Al Root,Tesla Stock Has Been on a Tear. The Reasons Wh...,Barrons,n/a,www.barrons.com,0.190674,Somewhat-Bullish,0.989159,0.190674,Somewhat-Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2786,Sila's Gleb Yushin Shows How Immigrants Can Ch...,https://www.forbes.com/sites/stuartanderson/20...,20221212T050627,Stuart Anderson,"Gleb Yushin, an international student to Ameri...",Forbes,Business,www.forbes.com,0.325096,Somewhat-Bullish,0.018935,0.0,Neutral
2787,Li Auto ( LI ) Q3 2022 Earnings Call Transcript,https://www.fool.com/earnings/call-transcripts...,20221209T200020,Motley Fool Transcribing,LI earnings call for the period ending Septemb...,Motley Fool,n/a,www.fool.com,0.276606,Somewhat-Bullish,0.013557,-0.015396,Neutral
2788,Learning From Successful Investors,https://www.fool.com/investing/2022/12/10/lear...,20221210T140000,Chris Hill,"David Rubenstein, author of ""How to Invest: Ma...",Motley Fool,n/a,www.fool.com,0.257082,Somewhat-Bullish,0.0088,0.126103,Neutral
2789,An Investor's Look at the Energy Picture,https://www.fool.com/investing/2022/12/10/an-i...,20221210T114500,Deidre Woollard,"Plus, we chat about how to get kids into the s...",Motley Fool,n/a,www.fool.com,0.151928,Somewhat-Bullish,0.008647,0.13931,Neutral


### Guardar df como archivo csv

In [6]:
nombre_archivo = "AlphaVantage_Scraping.csv"
df.to_csv(nombre_archivo, index=False)